## L1
### PydanticOutputParser

講座の中では、一階層のJsonをパースする方法が出てきました。
他にも、Pydanticを用いて、複雑なJsonを出力する方法があります。

In [2]:
import json
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field
from typing import List
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
chat = ChatOpenAI(temperature=0.0)

今回は、食材の栄養と相性のよい食材を以下の形式で出力する方法を試しました。

```json
{
    "name": "str: 食材名",
    "nutrition": {
        "calories": "int: 100gあたりカロリー（kcal）",
        "carbohydrates": "float: 100gあたり炭水化物（g）",
        "protein": "float: 100gあたりたんぱく質（g）",
        "vitamin_c": "float: 100gあたりビタミンC（mg）",
        "fiber": "float: 100gあたり食物繊維（g）",
    }
}
```

ちなみに、以前自己流で指示していた方法はこれに従ってくださいというものでした。
まあまあの精度でできていましたが、OutputParserとFunction callingがある今、使うことはなさそうです。。

以下のようにPydanticでクラス定義し、PydanticOutputParserを作ります。
簡単ですね。

In [3]:
class Nutrition(BaseModel):
    calories: int = Field(description='100gあたりカロリー（kcal）')
    carbohydrates: float = Field(description='100gあたり炭水化物（g）')
    protein: float = Field(description='100gあたりたんぱく質（g）')

    vitamin_c: float = Field(description='100gあたりビタミンC（mg）')
    fiber: float = Field(description='100gあたり食物繊維（g）')

class Food(BaseModel):
    name: str = Field(description='食材名')
    nutirition: Nutrition = Field(description='食材の100gあたり栄養素')
    compatible_foods: List[str] = Field(description='相性の良い食材（5つまで）')

parser = PydanticOutputParser(pydantic_object=Food)

ここからは講座の例と同様です。

In [4]:
about_food_template = """
{food}について教えてください。相性の良い食材や100gあたりの栄養成分など
{format_instructions}
"""
prompt = ChatPromptTemplate.from_template(template=about_food_template)

試しに白菜の情報を聞いてみました。

In [5]:
food = '白菜'
messages = prompt.format_messages(
    food=food, format_instructions=parser.get_format_instructions()
)
response = chat(messages)
food_data = parser.parse(response.content)

In [6]:
print(json.dumps(food_data.dict(),indent=2,ensure_ascii=False))

{
  "name": "白菜",
  "nutirition": {
    "calories": 13,
    "carbohydrates": 2.2,
    "protein": 1.2,
    "vitamin_c": 45.0,
    "fiber": 1.2
  },
  "compatible_foods": [
    "豚肉",
    "鶏肉",
    "牛肉",
    "エビ",
    "イカ"
  ]
}


いい感じに出力されました。相性の良い食材は、５つまでの指定を外すともっと出してくれます。